In [2]:
import json 

In [3]:
!ls

AAAMLP.pdf          create_data.py      data.ipynb          extracted_data.json


In [4]:
with open('extracted_data.json','r') as w:
    data = json.load(w)

In [5]:
print(data['Page40'])

Approaching (Almost) Any Machine Learning Problem 
 40     p = precision(y_true, temp_prediction)     r = recall(y_true, temp_prediction)     precisions.append(p)     recalls.append(r) ═════════════════════════════════════════════════════════════════════════  Now, we can plot these values of precisions and recalls.  ═════════════════════════════════════════════════════════════════════════ plt.figure(figsize=(7, 7)) plt.plot(recalls, precisions) plt.xlabel('Recall', fontsize=15) plt.ylabel('Precision', fontsize=15) ═════════════════════════════════════════════════════════════════════════  Figure 2 shows the precision-recall curve we get this way. 
 Figure 2: precision-recall curve  This precision-recall curve looks very different from what you might have seen on the internet. It’s because we had only 20 samples, and only 3 of them were positive samples. But there’s nothing to worry. It’s the same old precision-recall curve.  



## Let us clean the data
1. remove extra '==' and keep only '=='
2. remove extra white spaces 

In [6]:
import re
def clean_string(input_string):
    
    # Remove extra '==' or '...'
    input_string = re.sub(r'═{2,}', '==', input_string)
    
    # Remove extra white spaces
    input_string = ' '.join(input_string.split())
    
    return input_string

In [7]:
for i,j in data.items():
    data[i] = (clean_string(data[i]))

# Create Embeddings 

making use of sentence_transformers

using `multi-qa-mpnet-base-dot-v1` as it was tuned for semantic search

In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

In [15]:
text_to_encode = list(data.values())

In [16]:
len(text_to_encode)

300

In [17]:
text_embeddings = model.encode(text_to_encode)

In [18]:
text_embeddings.shape

(300, 768)

# Create Faiss Index

In [19]:
import faiss
import numpy as np

text_ids = [str(i) for i in range(len(text_to_encode))] 

# Step 2: Create a Faiss index using these embeddings
d = text_embeddings.shape[1]  # Dimension of the embeddings
index = faiss.IndexFlatL2(d)  # L2 distance metric for the index
index.add(text_embeddings)



faiss.write_index(index, index_file)
print(f"Index saved to {index_file}")

Index saved to faiis_index_book.index


#### we have what we need now let us build the app 

In [ ]:
# query_keyword = "example_keyword"  # Replace with your own keyword

# query_embedding = np.random.rand(1, 128).astype('float32')

# # Perform the nearest neighbor search
# k = 5  # Number of nearest neighbors to retrieve
# distances, indices = index.search(query_embedding, k)

# # Print the nearest neighbors
# print("Query Keyword:", query_keyword)
# for i in range(k):
#     print(f"Nearest Neighbor {i + 1} - ID: {ids[indices[0][i]]}, Distance: {distances[0][i]}")


In [3]:
# read index

import faiss
import json


In [4]:

with open('extracted_data.json','r') as w:
    data = json.load(w)
    
text_to_search = list(data.values())


In [20]:
index_file = "faiis_index_book.index"
book_index = faiss.read_index(index_file)
import json 
with open('extracted_data.json','r') as w:
    data = json.load(w)

text_to_search = list(data.values())

def get_search(query, topK):
    query_embedding = embeddings_model.encode([query])
    distances, indices = book_index.search(query_embedding, topK)
    res = [text_to_search[i] for i in indices[0]]
    return "\n".join(res)

In [22]:
get_search('Show me a code to build logisitic regression ?', 5 )

'Approaching (Almost) Any Machine Learning Problem \n 184   Model Optimize Range of values Linear Regression  - fit_intercept - normalize   - True/False - True/False Ridge  - alpha - fit_intercept - normalize   - 0.01, 0.1, 1.0, 10, 100 - True/False - True/False k-neighbors  - n_neighbors - p  - 2, 4, 8, 16 …. - 2, 3  SVM  - C - gamma - class_weight   - 0.001,0.01..10..100..1000 - ‘auto’, RS* - ‘balanced’ , None Logistic Regression  - Penalty - C   - l1 or l2 - 0.001, 0.01…..10...100  Lasso  - Alpha - Normalize  - 0.1, 1.0, 10 - True/False  Random Forest  - n_estimators - max_depth - min_samples_split - min_samples_leaf - max features  - 120, 300, 500, 800, 1200 - 5, 8, 15, 25, 30, None - 1, 2, 5, 10, 15, 100 - 1, 2, 5, 10 - log2, sqrt, None  XGBoost  - eta - gamma - max_depth - min_child_weight - subsample - colsample_bytree - lambda - alpha   - 0.01,0.015, 0.025, 0.05, 0.1 - 0.05-0.1,0.3,0.5,0.7,0.9,1.0 - 3, 5, 7, 9, 12, 15, 17, 25 - 1, 3, 5, 7 - 0.6, 0.7, 0.8, 0.9, 1.0 - 0.6, 0.7, 0